# Notebook for Evan - Generating Satellite Releases
Code Author: Sahar El Abbadi
Date: 2023-04-04


In [1]:
# Setup

# imports
import pandas as pd
import datetime
from methods_source import calc_average_release

## Satellite release dataframe
Create satellite release summary file with timestamps as datetime objects in UTC. The start and stop times of the relases will be used as inputs to the calc_average_releases function.

Dummy dataset includes one example time period when we have no release record (e.g. satellite release occurred on a weekend, and there is no meter file for the date in question), and one release where we have a metered data file for the time period in question.

In [2]:
# Dummy dataframe to be filled with satellite data
# Use UTC for start and end timestamps
satellite_timestamps = pd.DataFrame({
    'release_id': [1, 2],
    'satellite': ['Test1', 'Test2'],
    'team_analyzing': ['Luis', 'Maxar']
    'start_release':[datetime.datetime(2022, 10, 15, 18, 16, 10), datetime.datetime(2022, 10, 25, 17, 44, 22)],
    'end_release': [datetime.datetime(2022, 10, 15, 18, 36, 00), datetime.datetime(2022, 10, 25, 18, 10, 00)],
})

## Calculate the gas flow rate during release periods

Use calc_average_releases function to generate gas flow profile for each time period of interest. Here, I apply the function to each row in the satellite_releases dataframe, although other methods may be used as well.

Function inputs:
- start_t: datetime object in UTC
- stop_t: datetime object in UTC

Notes:
- Function will return an error message if start_t and stop_t are on different dates (ie you cannot calculate the average flow rate if they straddle different days of testing)
- Function will return an error message if start_t is chronologically after stop_t
- If start_t and end_t occur on a date for which we do not have metered data, function will output values for a zero release (flow rate of 0 kgh, uncertainty of 0, meter and gas composition of np.nan)

Function outputs a Python dictionary with the following keys:
- gas_kgh_mean: mean gas flow rate over the time period of interest, as whole gas
- gas_kgh_sigma: standard deviation of the meter reading over the period of interest, as whole gas. This value represents the physical variability in the flow rate.
- meter_sigma: the standard deviation calculated based on the uncertainty in the meter reading for the flow rate during the period in question. Emerson reports uncertainty as a percentage of flow rate, and the function converts this value to a sigma value with units of kgh whole gas
- ch4_fraction_{gas_comp_source}: the fraction methane based on the input source for gas composition. Gas composition can be 'su_normalized', 'su_raw', or 'km'. See other documentation for additional details. This value is averaged over the time period of interest, although unless the trailer was changed mid-release, value is expected to be constant
- ch4_fraction{gas_comp_source}_sigma: the sigma value associated with the gas composition, representative of the uncertainty associated with measurements of methane mole fraction.
- ch4_kgh_mean: mean methane flow rate, calculated from the gas flow rate and methane mole fraction
- cg4_kgh_sigma: total uncertainty in the methane flow rate. This value combines physical variability in gas flow rate (gas_kgh_sigma) with uncertainty in the meter reading (meter_sigma) and uncertainty in gas composition (ch4_fraction_{gas_comp_source}_sigma.

In [3]:
# Run function to determine flow rates for each release

release_rates =[]

for index, release in satellite_timestamps.iterrows():
    start_t = release.start_release
    end_t = release.end_release
    release_details = calc_average_release(start_t, end_t)
    release_rates.append(release_details)

final_releases = pd.DataFrame(release_rates)
summary = pd.concat([satellite_timestamps, final_releases], axis=1)

print(summary)

   release_id satellite       start_release         end_release  gas_kgh_mean  \
0           1     Test1 2022-10-15 18:16:10 2022-10-15 18:36:00      0.000000   
1           2     Test2 2022-10-25 17:44:22 2022-10-25 18:10:00   1025.589677   

   gas_kgh_sigma  meter_sigma  ch4_fraction_km  ch4_fraction_km_sigma  \
0       0.000000          NaN              NaN                    NaN   
1      99.868814     1.310053         0.950644               0.001366   

   ch4_kgh_mean  ch4_kgh_sigma  
0           NaN            NaN  
1    974.970673      94.958189  
